In [227]:
import Bio
import pandas as pd
from Bio.Seq import Seq
from Bio.SeqUtils import gc_fraction, MeltingTemp as mt
from collections import Counter
import random
import time

In [23]:
# generate as many umis as possible
# define the length
umilen = 16

# function to create umis
def create_umi(length = umilen):
    return ''.join(random.choices('ACGT', k=length))

# for a sequence of 16 nucleotides, the average count for nucleotide must be 4
# on average, counts should be 4!
# we can set 1 as the highest deviation from the average that we accept for the umi to be "equilibrated"
threshold =  1

# function to check if nucleotides are in equilibrium
def count_nns(umi, threshold):
    counts = Counter(umi).values()
    mean_count = sum(counts) / len(counts)
    
    # calculate deviations
    devs = [abs(count - mean_count) for count in counts]
    # check that all counts are within the deviation threshold
    if all(dev <= threshold for dev in devs):
        return umi
        
random.seed(666)

In [ ]:
# we have randomly chosen umis that are roughly equilibrated
# now we need to check that the umi doesn't have more than N of the same nucleotide together

In [124]:
# homopolymers no longer than 0 == no homopolymers
N = 0
# number of homopolymers threshold
P = 0

def check_homopolymers(umiseq, N, P):
    # create empty variables to store nucleotides
    current_nn = None
    conseq = 0 # initiate the count of nucleotides from zero
    homseq = 0 # count for homopolymers
    longest = 0 # keeps track of the longest homopolymer

    # iterate through the umi sequence
    for nn in umiseq:
        if nn == current_nn:
            conseq += 1 # add one to the count if subsequent nucleotides after current are the same
            homseq += 1
            if conseq >= longest:
                longest = conseq
        else:
            # replace the variable with the next nucleotide if it is different
            conseq = 0 # resets the nucleotide count
        current_nn = nn  # Change to a new nucleotide

    # check if the homopolymer count exceeds the N threshold
    if homseq > P: # has at least one homopolymer
        return False  # Homopolymer of length greater than N found
    # filter by longest homopolymer
    elif longest > N:
        return False
    else:
        return umiseq  # No homopolymer longer than N found

In [82]:
# now, we need to check that the sequence is not complementary or within the target sequence 

def check_seq_match(umi, target):
    rev_target = Seq(target)[::-1]  # reverse target sequence
    rev_comp_target = Seq(target).reverse_complement()  # reverse complement target

    # check the sequences
    if str(umi) == str(target):
        return False, 'UMI is the same as the target sequence'
    elif str(umi) == str(rev_target):
        return False, 'UMI is the reverse of the target sequence'
    elif str(umi) == str(rev_comp_target):
        return False, 'UMI is the reverse complement of the target sequence'

    return umi, 'UMI is unique and does not match the target sequence'

In [ ]:
# but we also want to check for partial matches

In [182]:
def check_partial_matches(umi, target, min_match = 6):
    # Generate reverse and reverse complement of the target sequence
    rev_target = Seq(target)[::-1]  # Reverse of the target sequence
    rev_comp_target = Seq(target).reverse_complement()  # Reverse complement using BioPython
    
    # Function to check for partial matches in a given target sequence
    def find_partial_matches(umi, target, min_match):
        umi_len = len(str(umi))
        target_len = len(str(target))

        # function to iterate through nucleotides from zero to the target's length minus match length plus one
        # that is the last kmer of length min_match + 1
        for i in range(target_len - min_match + 1):
            # iterates through nucleotides from the first kmer to the length of the umi plus +
            for j in range(min_match, umi_len + 1):
                # checks if umi in target and vice versa
                if target[i:i+j] in umi or umi in target[i:i+j]:
                    return 'Partial match', target[i:i+j] # there's a match
        # this umi is returning the sequence
        return 'No match of length %s'%(min_match), 'comparing: %s'%(target[i:i+j])
    
    # check partial matches of umi on the target, reverse, and reverse complement
    # using the iterating function above
    for seq in [target, str(rev_target), str(rev_comp_target)]:
        umitemp, match_sequence = find_partial_matches(umi, seq, min_match)
        # if there is a match, the match_sequence string will contain text and the matching sequences
        # thus, it would be longer than the minimum match length allowed
        if len(match_sequence) > min_match:
            return False, f'Partial match found: {match_sequence} in sequence {seq}'
    return umi, 'No significant partial matches found'

In [183]:
# test the tree functions
test = 'AAACCCTTTGGGATCG'
test1 = 'TACGTACGTACGTACG'

target = 'AAACCCTTTGGGATCG'
print(check_homopolymers(test, N, P))
print(check_homopolymers(test1, N, P))

print(check_seq_match(test,target))
print(check_seq_match(test1,target))

print(check_partial_matches(test,target, 5))
print(check_partial_matches(test1,target, 5))

umipass, statement = check_partial_matches(test1,target, 5)
print(umipass)

False
TACGTACGTACGTACG
(False, 'UMI is the same as the target sequence')
('TACGTACGTACGTACG', 'UMI is unique and does not match the target sequence')
(False, 'Partial match found: comparing: CCAAA in sequence GCTAGGGTTTCCCAAA')
(False, 'Partial match found: comparing: GATCG in sequence AAACCCTTTGGGATCG')
False


In [181]:
target = 'CACACCTCGGTGTGAAGCAAATGATTGTTGCTATTAACAAGATGGACGACAAATCTGTCAACTGGGCACAATCTAGGTACGATGAAATAGTTAAGGAAGTATCCTCCTTTGTCAAGAAGATCGGCTACAACCCTGAGAAGATCCCGTTCGTCCCCATTTCTGGTTGGCACGGAGATAACATGCTCGAGAAGTCATCTAACCTCTCATGGTATAAAGGCCCCACATTACTCGAGGCCCTCGACTCTGTGTCAGAACCCAAGAGACCAACGGAAAAGCCCCTCCGAATTCCCCTTCAAGATGTTTACAAGATTGGAGGTATTGGAACTGTGCCTGTAGGCCGTGTTGAGACTGGGGTTCTCAAACCAGGAATGAACGTTACTTTCTCCCCTGCTGGTTTGACCACTGAAGTTAAGTCTGTTGAGATGCACCACGTCTCTCTCCCTGAGGCTGTCCCAGGTGATAACGTTGGTTTCAATGTCAAGAATCTGTCAGTTAAGGATATTCGTCGTGGTATGGTTGCTGGTGATGCCAAGAATGATCCCCCTCAAGAGACTGAAGATTTTAATGCCCAAGTTATTATTCTCAATCACCCTGGTCAGATCCATGCAGGATATGCCCCAGTGCTTGATTGTCACACTGCTCATATCGCCTGTAAGTTCAGCGAGATTCTCTCCAAAGTAGATCGTAGATCTGGTCAAGAGACTGAGGCTGCCCCTAAGAATATCAAGAACGGAGATGCCCGCCATAGTTAAACTCACTCCCTCCCAAGGCCCATGTGTGTGGAGTCTTTCTCTGATTACCCACCCCCTTGG'
testlist = ['CTGCTGCAGAATACAT', 'TACACTGCGCAGCTAT', 'CAGGAGTAACTACTGC', 'GTACGCCCAGCATATG', 'CGTATCTAGGTAACAT', 'TGATGGACTCACCAGC', 'GTCTGCGTGATTAACA', 'GAACTAGCTGCAGGCT', 'TATGCTAGACAGGTCA', 'GAGTACATCGCTCAGT']

newlist = []
for umi in testlist:
    check_seq_match(umi, target)
    umibool, statement = check_seq_match(umi, target)
    # print('step1', umibool, statement)
    if umibool != False: 
        # print('step2', umibool, statement)
        if umibool not in newlist and umibool != True:
            # print('umibool', umibool)
            newlist.append(umibool)
            
print('Finding UMIs with no full match:')
print('Remaining UMIs %s'%(len(newlist)))
print(newlist[0:10], '\n')


newlist1 = []
for umi in newlist:
    umibool, statement = check_partial_matches(umi, target, min_match = 6)
    # print('step1', umibool, statement)
    if umibool != False:
        # print('step2', umibool, statement)
        if umibool not in newlist1:
            # print('umibool', umibool)
            newlist1.append(umibool)
            
print('UMIs with no partial match')
print('Remaining UMIs %s'%(len(newlist1)))
print(newlist1[0:10], '\n')


Finding UMIs with no full match:
Remaining UMIs 10
['CTGCTGCAGAATACAT', 'TACACTGCGCAGCTAT', 'CAGGAGTAACTACTGC', 'GTACGCCCAGCATATG', 'CGTATCTAGGTAACAT', 'TGATGGACTCACCAGC', 'GTCTGCGTGATTAACA', 'GAACTAGCTGCAGGCT', 'TATGCTAGACAGGTCA', 'GAGTACATCGCTCAGT'] 

UMIs with no full match
Remaining UMIs 2
['CTGCTGCAGAATACAT', 'TGATGGACTCACCAGC'] 



In [184]:
# implement the tree functions
# assume the following target barcode in Fungi
# TEF1α region Al33F (5′-GAYTTCATCAAGAACATGAT-′3) and Al33R (5′-GACGTTGAADCCRACRTTGTC-′3)
# from https://cdnsciencepub.com/doi/10.1139/gen-2018-0083
# create umis

# Trichia persimili, a myxomicetes (group found in deadwood, doesnt mean this spp is saproxylic)
# GenBank: FJ546692.1
target = 'CACACCTCGGTGTGAAGCAAATGATTGTTGCTATTAACAAGATGGACGACAAATCTGTCAACTGGGCACAATCTAGGTACGATGAAATAGTTAAGGAAGTATCCTCCTTTGTCAAGAAGATCGGCTACAACCCTGAGAAGATCCCGTTCGTCCCCATTTCTGGTTGGCACGGAGATAACATGCTCGAGAAGTCATCTAACCTCTCATGGTATAAAGGCCCCACATTACTCGAGGCCCTCGACTCTGTGTCAGAACCCAAGAGACCAACGGAAAAGCCCCTCCGAATTCCCCTTCAAGATGTTTACAAGATTGGAGGTATTGGAACTGTGCCTGTAGGCCGTGTTGAGACTGGGGTTCTCAAACCAGGAATGAACGTTACTTTCTCCCCTGCTGGTTTGACCACTGAAGTTAAGTCTGTTGAGATGCACCACGTCTCTCTCCCTGAGGCTGTCCCAGGTGATAACGTTGGTTTCAATGTCAAGAATCTGTCAGTTAAGGATATTCGTCGTGGTATGGTTGCTGGTGATGCCAAGAATGATCCCCCTCAAGAGACTGAAGATTTTAATGCCCAAGTTATTATTCTCAATCACCCTGGTCAGATCCATGCAGGATATGCCCCAGTGCTTGATTGTCACACTGCTCATATCGCCTGTAAGTTCAGCGAGATTCTCTCCAAAGTAGATCGTAGATCTGGTCAAGAGACTGAGGCTGCCCCTAAGAATATCAAGAACGGAGATGCCCGCCATAGTTAAACTCACTCCCTCCCAAGGCCCATGTGTGTGGAGTCTTTCTCTGATTACCCACCCCCTTGG'

umi_raw = []
# to generate 96 forward and 96 reverse umis
# and check for nucleotide composition balance
# 4**16 is as many combinations as we can get =  4294967296
# 100000 is the number of initial random tags, you can reduce or increase the number if it takes too long
# or if no enough umis pass the test
while len(umi_raw) <= 100000:
    umi = create_umi()
    umi_pass1 = count_nns(umi, threshold)
    
    # add the umi to the list if it is not already there
    if umi_pass1 not in umi_raw and umi_pass1 != None:
        umi_raw.append(umi_pass1)
print('Raw UMIs created: %s'%(len(umi_raw)))
print(umi_raw[0:10], '\n')

# check umis for homopolymers

N = 2 # allow the longest homopolymer to have two nucleotides
P = 2 # allow a max of one homopolymer

umi_uniq = []

for umi in umi_raw:
    if check_homopolymers(umi, N, P) is False:
        pass
    else:
        if umi not in umi_uniq:
            umi_uniq.append(umi)
print('Non-homopolymer UMIs with homopolymer number threshold %s and nucleotide count threshold %s'%(P,N))
print('Remaining UMIs %s'%(len(umi_uniq)))
print(umi_uniq[0:10], '\n')

# check for matches
umi_nofmatch = []

for umi in umi_uniq:
    check_seq_match(umi, target)
    umibool, statement = check_seq_match(umi, target)
    if umibool != False: 
        if umibool not in umi_nofmatch and umibool != True:
            umi_nofmatch.append(umibool)
print('UMIs with no full match')
print('Remaining UMIs %s'%(len(umi_nofmatch)))
print(umi_nofmatch[0:10], '\n')

# check for partial matches
umi_preselected = []

for umi in umi_nofmatch:
    umibool, statement = check_partial_matches(umi, target, min_match = 6)
    if umibool != False:
        if umibool not in umi_preselected:
            umi_preselected.append(umibool)
            
print('UMIs with no partial match')
print('Remaining UMIs %s'%(len(umi_preselected))) # number of umis that pass all tests
umi_preselected[0:10] # prints the first 10 in the list

Raw UMIs created: 1000001
['TTGATCGAGCCACTGG', 'GGTAAAACACGTCGTT', 'ATGAGTAGATCCCCGA', 'ACATAAGTTCAGGTCC', 'CTAAAGGGTCTTCGTG', 'TAATCCGCAAGGGAGT', 'TCGTCTAGCTCGAAAA', 'TCAACGGCTAGCGGTC', 'AATACTCCCTGAGGAG', 'TATCAGGCCGAAACTC'] 

Non-homopolymer UMIs with homopolymer number threshold 2 and nucleotide count threshold 2
Remaining UMIs 351031
['CGATATGAGCAATGCC', 'TATCATGATCTCAGGA', 'GCACTCATGTGAGCAG', 'AGACTCTGACTTGTAA', 'GGACGTGTACATCATG', 'ATAGCTGCTGTTCACA', 'ACGCGTCTGAAGTAGT', 'GAGGTGCAGATACTTC', 'TTAGTCTAGTCAAGCA', 'AGACCGGTACTAGATG'] 

UMIs with no full match
Remaining UMIs 351031
['CGATATGAGCAATGCC', 'TATCATGATCTCAGGA', 'GCACTCATGTGAGCAG', 'AGACTCTGACTTGTAA', 'GGACGTGTACATCATG', 'ATAGCTGCTGTTCACA', 'ACGCGTCTGAAGTAGT', 'GAGGTGCAGATACTTC', 'TTAGTCTAGTCAAGCA', 'AGACCGGTACTAGATG'] 

UMIs with no partial match
Remaining UMIs 142435


['TATCATGATCTCAGGA',
 'GCACTCATGTGAGCAG',
 'AGACTCTGACTTGTAA',
 'ATAGCTGCTGTTCACA',
 'TTAGTCTAGTCAAGCA',
 'GACACACCGTTAGTGT',
 'AACTCGTCATGTGGTA',
 'TCATGTACCTGACGAC',
 'AATGCATACTCGAGCG',
 'CAGTGTGTCAACACGG']

In [261]:
# finally, calculate GC content and annealing temperature
# for the remaining 142435 umis (we need 96*2)
# we can check the first 1000 or so umis
# we will select those with compatible temperatures and good CG content

padd = 'GGTAG'
primer_forward = 'GAYTTCATCAAGAACATGAT'
primer_reverse = 'GACGTTGAADCCRACRTTGTC'
gc_min = 46
gc_max = 54
# based on https://www.tandfonline.com/doi/full/10.1080/15572536.2006.11832842#d1e421 - FUNGI!
temp_min = 55
temp_max = 67

# how many tagged primers to generate
# e.g. if num_primers = 96, it will generate 96 forward and 96 reverse
num_primers = 96

# creates the table to store information
tagged_primers = pd.DataFrame(columns = ['sense','padd_seq','umi_seq','primer_seq','full_primer','gc_content','temp'])

# to make sure no same umi is used for two primer pairs:
umi_track = []

# initiate counters to generate just the primers necessary
f_count = 0
for umi in umi_preselected: # umi_preselected[0:20000] if you want to test things fast, you can select the first e.g. 20000 rows
    if umi not in umi_track:
        fullseq = padd + umi + primer_forward
        # print(fullseq)
        gc_content = gc_fraction(fullseq) * 100
        # print(gc_content)
        temp = mt.Tm_NN(fullseq)
        # print(temp)
        sense = 'forward' # python is zero indexed

        # add thresholds if you wanna keep padd+umi+primer temperatures that overlap the
        # PCR protocol annealing temperatures already
        # and to keep the GC content close to 50% - second filter for that
        # greater than or equal AND lesser then or equal to
        if gc_min <= gc_content <= gc_max and temp_min <= temp <= temp_max:
            row = [sense, padd, umi, primer, fullseq, gc_content, temp]
            tagged_primers.loc[len(tagged_primers)] = row # appends the row
            umi_track.append(umi)
            # add to the counters
            f_count += 1
            # print('passed %s'%(f_count))
            if f_count == num_primers:
                break
print('done with forward primers')
        
r_count = 0
for umi in umi_preselected: # umi_preselected[0:20000] if you want to test things fast, you can select the first e.g. 20000 rows
    if umi not in umi_track:
        fullseq = padd + umi + primer
        gc_content = gc_fraction(fullseq) * 100
        temp = mt.Tm_NN(fullseq)
        sense = 'reverse' # python is zero indexed
        
        if gc_min <= gc_content <= gc_max and temp_min <= temp <= temp_max:
            row = [sense, padd, umi, primer, fullseq, gc_content, temp]
            tagged_primers.loc[len(tagged_primers)] = row # appends the row
            umi_track.append(umi)
            r_count += 1
            if r_count == num_primers:
                break
    
print('done with reverse primers')
            
print('Umis generated under the thresholds: %s'%(len(tagged_primers)))
print('Of which %s are forward and %s are reverse'%(len(tagged_primers[tagged_primers['sense'] == 'forward']),
                                                    len(tagged_primers[tagged_primers['sense'] == 'reverse'])))
tagged_primers.head(10)

done with forward primers
done with reverse primers
Umis generated under the thresholds: 192
Of which 96 are forward and 96 are reverse


,sense,padd_seq,umi_seq,primer_seq,full_primer,gc_content,temp
0,forward,GGTAG,GCGATTGCCTGCACAG,GAYTTCATCAAGAACATGAT,GGTAGGCGATTGCCTGCACAGGAYTTCATCAAGAACATGAT,47.5,66.610557
1,forward,GGTAG,GTCGCCCAGCATGTGA,GAYTTCATCAAGAACATGAT,GGTAGGTCGCCCAGCATGTGAGAYTTCATCAAGAACATGAT,47.5,66.160343
2,forward,GGTAG,GACGGACGCAGTCTCT,GAYTTCATCAAGAACATGAT,GGTAGGACGGACGCAGTCTCTGAYTTCATCAAGAACATGAT,47.5,65.667593
3,forward,GGTAG,GCGCGACTTGAGATCC,GAYTTCATCAAGAACATGAT,GGTAGGCGCGACTTGAGATCCGAYTTCATCAAGAACATGAT,47.5,66.163848
4,forward,GGTAG,CGAAGACTCGCGTCGT,GAYTTCATCAAGAACATGAT,GGTAGCGAAGACTCGCGTCGTGAYTTCATCAAGAACATGAT,47.5,66.368894
5,forward,GGTAG,GTACGGCAGACTGTCC,GAYTTCATCAAGAACATGAT,GGTAGGTACGGCAGACTGTCCGAYTTCATCAAGAACATGAT,47.5,65.678127
6,forward,GGTAG,CGAGTCTTCAGCGACG,GAYTTCATCAAGAACATGAT,GGTAGCGAGTCTTCAGCGACGGAYTTCATCAAGAACATGAT,47.5,66.109110
7,forward,GGTAG,CGGCACAGAGTTCTGC,GAYTTCATCAAGAACATGAT,GGTAGCGGCACAGAGTTCTGCGAYTTCATCAAGAACATGAT,47.5,66.543285
8,forward,GGTAG,GCATGAGGCACTCGTC,GAYTTCATCAAGAACATGAT,GGTAGGCATGAGGCACTCGTCGAYTTCATCAAGAACATGAT,47.5,66.152573
9,forward,GGTAG,CTCGCTGTAGAGCCAG,GAYTTCATCAAGAACATGAT,GGTAGCTCGCTGTAGAGCCAGGAYTTCATCAAGAACATGAT,47.5,65.728494


In [265]:
# save the file!
# TEF1α region Al33F (5′-GAYTTCATCAAGAACATGAT-′3) and Al33R (5′-GACGTTGAADCCRACRTTGTC-′3)
tagged_primers.to_csv('./TEFa_Al33F-Al33R_tagged.csv', sep = '\t', index_label = 'index')